<div class='alert alert-info'>
    <strong> <h3>Projet d'estimation de potentiel par offre OA : 05/12/2022</h3> </strong>
</div>

In [1]:
## Import des packages(bibliothèques) à utiliser
from tqdm import tqdm
import datetime
import pandas as pd
import csv
import numpy as np
import os
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<div class='alert alert-danger'>
    <strong> <h5>Import des fichiers sources</h5> </strong>
</div>

<div class='alert alert-info'>
    <strong> <h5>Import fichier "Contact"</h5> </strong>
</div>

In [2]:
# Import fichier "Contact"
df_contact=pd.read_csv(r"P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Input_Files\Contact_Act.csv",
                       delimiter='|', engine="python", dtype={'NOIDE':str}, encoding='mbcs')

In [3]:
df_contact

,NOIDE,AVEC_MAIL,AVEC_TEL_FIX,AVEC_TEL_PORT,AVEC_ADRESSE,NAT_ACTIVITE
0,0102501776,NaN,NaN,Tel_Port,Adresse,Retraité
1,0103311214,MAIL,Tel_Fix,Tel_Port,Adresse,Actif
2,0104662172,MAIL,Tel_Fix,Tel_Port,Adresse,Actif
3,0105031338,MAIL,NaN,Tel_Port,Adresse,Actif
4,0105144256,MAIL,Tel_Fix,NaN,Adresse,Actif
...,...,...,...,...,...,...
7990359,0103126657,MAIL,Tel_Fix,Tel_Port,Adresse,NaN
7990360,0106869908,NaN,NaN,NaN,Adresse,NaN
7990361,0104551936,NaN,NaN,NaN,Adresse,NaN
7990362,0106270773,NaN,NaN,NaN,Adresse,NaN


In [4]:
## Vérifier si doublons des NOIDE
fichier_verif_doublon=df_contact.copy() 
nrow=df_contact.shape[0]  # nrow=nbre de lignes di dataframe 'df_contact' et shape(0) --> 0 = prendre 1er élément du shape
fichier_verif_doublon['Count']=pd.Series(1,range(nrow))  # Créer la colonne 'Count' dans dataframe 'fichier_verif_doublon' et mettre la valeur '1' autant de lignes de valeur de nrow
fichier_verif_doublon_2=fichier_verif_doublon.groupby('NOIDE')['Count'].sum().reset_index()
NOIDE_doublon_1=fichier_verif_doublon_2[fichier_verif_doublon_2['Count']>1]
NOIDE_doublon_1.head()

,NOIDE,Count


<div class='alert alert-info'>
    <strong> <h5>Recodifier les différents flags de canal (Mail, Tél, corrier) de contact de dataframe "Contact"</h5> </strong>
</div>

In [5]:
## Calculer les effectifs de chaque flag de canal de contact
liste=['AVEC_MAIL', 'AVEC_TEL_FIX', 'AVEC_TEL_PORT','AVEC_ADRESSE', 'NAT_ACTIVITE']
for elem in liste[:]:
    display(pd.DataFrame(df_contact[elem].value_counts(sort=True,ascending=True,dropna=False)))

,AVEC_MAIL
MAIL,3713958
NaN,4276406


,AVEC_TEL_FIX
Tel_Fix,2967415
NaN,5022949


,AVEC_TEL_PORT
Tel_Port,3382333
NaN,4608031


,AVEC_ADRESSE
NaN,412595
Adresse,7577769


,NAT_ACTIVITE
Service Volontaire,15
TNS et Agricole,12665
Sans activité,241564
Scolarité,438534
Retraité,1491238
Actif,2190187
NaN,3616161


In [4]:
# Remplacer les cellules renseignées de chaque flag (ex: MAIL, Tel_Fix....) par 1 et 0 si les cellules sont non renseignées (NaN)
def contact(x):
    if type(x)==str:
        return 1
    else:
        return 0
    
colon=df_contact.columns[1:-1]

for col in colon:
    
    df_contact[col]=df_contact[col].apply(contact)

In [7]:
df_contact

,NOIDE,AVEC_MAIL,AVEC_TEL_FIX,AVEC_TEL_PORT,AVEC_ADRESSE,NAT_ACTIVITE
0,0102501776,0,0,1,1,Retraité
1,0103311214,1,1,1,1,Actif
2,0104662172,1,1,1,1,Actif
3,0105031338,1,0,1,1,Actif
4,0105144256,1,1,0,1,Actif
...,...,...,...,...,...,...
7990359,0103126657,1,1,1,1,NaN
7990360,0106869908,0,0,0,1,NaN
7990361,0104551936,0,0,0,1,NaN
7990362,0106270773,0,0,0,1,NaN


In [8]:
## Calculer les effectifs de chaque flag de canal de contact après la recodification des colonnes de dataframe "Contact"
liste=['AVEC_MAIL', 'AVEC_TEL_FIX', 'AVEC_TEL_PORT','AVEC_ADRESSE', 'NAT_ACTIVITE']
for elem in liste[:]:
    display(pd.DataFrame(df_contact[elem].value_counts(sort=True,ascending=True)))

,AVEC_MAIL
1,3713958
0,4276406


,AVEC_TEL_FIX
1,2967415
0,5022949


,AVEC_TEL_PORT
1,3382333
0,4608031


,AVEC_ADRESSE
0,412595
1,7577769


,NAT_ACTIVITE
Service Volontaire,15
TNS et Agricole,12665
Sans activité,241564
Scolarité,438534
Retraité,1491238
Actif,2190187


<div class='alert alert-info'>
    <strong> <h5>Import fichier "Consentement"</h5> </strong>
</div>

In [5]:
# Import fichier "Consentement"
df_consentement=pd.read_csv(r"P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Input_Files\Consentement_Act.csv",
                           delimiter="|", engine="python", dtype={"NOIDE":str}, encoding="mbcs")

In [6]:
df_consentement

,NOIDE,CODTRT,LIBTRT,VALACD
0,0108953089,PROMOANAMAIL,Promotion d'offres et services analogues MGEN ...,True
1,0108903696,PROMOHANACOUR,Promotion d'autres offres et services MGEN pa...,True
2,0108676967,PROMOHANACOUR,Promotion d'autres offres et services MGEN pa...,True
3,0108708557,PROMOANASMS,Promotion d'offres et services analogues MGEN ...,True
4,0108922883,PROMOHANATEL,Promotion d'autres offres et services MGEN pa...,True
...,...,...,...,...
8755776,0108659036,PROMOHANACOUR,Promotion d'autres offres et services MGEN pa...,True
8755777,0103548427,PROMOHANACOUR,Promotion d'autres offres et services MGEN pa...,True
8755778,0107186086,PROMOANASMS,Promotion d'offres et services analogues MGEN ...,True
8755779,0101555429,PROMOANASMS,Promotion d'offres et services analogues MGEN ...,True


<div class='alert alert-info'>
    <strong> <h5>Pivoter le dataframe "Consentement"</h5> </strong>
</div>

In [7]:
df_consentement_mod=pd.pivot_table(df_consentement, index= "NOIDE", columns= "CODTRT")

df_consentement_mod.fillna(0,inplace=True)

## Transformer notre tableau croisé dynamique en dataframe

df_consentement_mod.columns=df_consentement_mod.columns.droplevel(0)

df_consentement_mod.columns.name = None

df_consentement_mod=df_consentement_mod.reset_index()

df_consentement_mod

,NOIDE,PROMOANACOUR,PROMOANAMAIL,PROMOANASMS,PROMOANATEL,PROMOHANACOUR,PROMOHANAMAIL,PROMOHANASMS,PROMOHANATEL
0,0100000001,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1,0100000518,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
2,0100001385,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
3,0100002413,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
4,0100002558,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1515467,0109408676,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1515468,0109408682,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1515469,0109408711,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1515470,0109408726,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0


In [9]:
## Vérifier si doublons des NOIDE
fichier_verif_doublon=df_consentement_mod.copy() 
nrow=df_consentement_mod.shape[0]  # nrow=nbre de lignes di dataframe 'df_consentement_mod' et shape(0) --> 0 = prendre 1er élément du shape
fichier_verif_doublon['Count']=pd.Series(1,range(nrow))  # Créer la colonne 'Count' dans dataframe 'fichier_verif_doublon' et mettre la valeur '1' autant de lignes de valeur de nrow
fichier_verif_doublon_2=fichier_verif_doublon.groupby('NOIDE')['Count'].sum().reset_index()
NOIDE_doublon_2=fichier_verif_doublon_2[fichier_verif_doublon_2['Count']>1]
NOIDE_doublon_2.head()

,NOIDE,Count


<div class='alert alert-info'>
    <strong> <h5>Import l'ensemble des fichiers d'équipements et consolider ensemble </h5> </strong>
</div>

In [10]:
# Import ensemble des fichiers équipements dans répertoire "Input_Files"
directory="P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Input_Files"

root, dirs, files = next(os.walk(directory))

In [11]:
# Fonction pour importer ensemble des fichiers de nom commencent par "Equip" dans répertoire "Input_Files"
liste_df=[]

for file in files:
    
    if file.startswith("Equip") or file.startswith("EQUIP"):
        
        df = pd.read_csv(r"P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Input_Files\\"+file,
                        delimiter='|', engine='python',encoding='mbcs',dtype={'NOIDE':str})
        
        print(f"Le fichier {file} est importé avec {df.shape[0]} lignes \n")
        
        df.rename(columns={'NAISSANCE':'DATNAI'},inplace=True)
        
        liste_df.append(df)
        
df1=liste_df[0]

for df in liste_df[1:]:
    
    df1=df1.merge(df,how='outer',on=['NOIDE','DATNAI'])
        
df_portefeuile_MSP=df1.copy()

Le fichier Equip_Assur_Vie.csv est importé avec 4633 lignes 

Le fichier Equip_Deces.csv est importé avec 48341 lignes 

Le fichier Equip_ent_cal_cau_hab.csv est importé avec 1815816 lignes 

Le fichier Equip_Obseques.csv est importé avec 74444 lignes 

Le fichier Equip_PIM.csv est importé avec 29003 lignes 

Le fichier Equip_Retraite.csv est importé avec 2433 lignes 



In [12]:
# Afficher les 20 dernières lignes du Dataframe 'df_portefeuile_MSP'
df_portefeuile_MSP.tail(20)

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,CODSEC,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE
1855399,0108301268,NaN,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855400,0108953243,NaN,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855401,0108485570,NaN,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855402,0104870824,NaN,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855403,0109505257,NaN,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855404,0109487268,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855405,0105751990,NaN,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855406,0109509711,NaN,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855407,0109531983,NaN,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1855408,0108933514,NaN,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [13]:
# Supprimer la colonne "CODSEC" du Dataframe "df_portefeuile_MSP"

df_portefeuile_MSP.drop('CODSEC',axis=1, inplace=True)

In [14]:
# Remplacer les cellules non renseignées (NaN) par 0 
# 0 --> il ne possède pas l'offre
# 1 --> il possède deja l'offre
df_portefeuile_MSP.fillna(0,inplace=True)

In [15]:
df_portefeuile_MSP

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE
0,0103544722,1.0,1992,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0105893027,1.0,1999,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0106217145,1.0,1981,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0107977127,1.0,1959,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0105891091,1.0,1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0.0,1994,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1855415,0105880759,0.0,1998,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1855416,0109511742,0.0,1997,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1855417,0104782983,0.0,1968,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
## Transformation les flags "TOP_équipement" en entier

df_portefeuile_MSP.rename(columns={"TOP_OBSEQUE":"TOP_OBS","TOP_PROTECT_JURID":"TOP_PROTECT_JURIQ"}, inplace=True)

df_portefeuile_MSP[['TOP_ASSUR_VIE', 'TOP_DECES', 'TOP_CAM','TOP_TOUT_HABITAT', 'TOP_HABITAT_MGEN', 'TOP_OBS', 'TOP_PIM',
        'TOP_RETRAITE']]=df_portefeuile_MSP[['TOP_ASSUR_VIE', 'TOP_DECES', 'TOP_CAM',
       'TOP_TOUT_HABITAT', 'TOP_HABITAT_MGEN', 'TOP_OBS', 'TOP_PIM',
        'TOP_RETRAITE' ,'TOP_PROTECT_JURIQ','TOP_SANTE_ANIM']].astype(int) 

In [17]:
df_portefeuile_MSP

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE
0,0103544722,1,1992,1,0,0,0,0,0,0
1,0105893027,1,1999,1,0,0,0,0,0,0
2,0106217145,1,1981,0,0,0,0,0,0,0
3,0107977127,1,1959,0,0,0,0,0,0,0
4,0105891091,1,1997,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,0,0,0,0,0,0,1
1855415,0105880759,0,1998,0,0,0,0,0,0,1
1855416,0109511742,0,1997,0,0,0,0,0,0,1
1855417,0104782983,0,1968,0,0,0,0,0,0,1


<div class='alert alert-info'>
    <strong> <h5>Contrôle l'unicité des NOIDE du fichier d'équipements consolidé</h5> </strong>
</div>

In [18]:
## Vérifier si doublons des NOIDE
fichier_verif_doublon=df_portefeuile_MSP.copy() 
nrow=df_portefeuile_MSP.shape[0]  # nrow=nbre de lignes di dataframe 'df_portefeuile_MSP' et shape(0) --> 0 = prendre 1er élément du shape
fichier_verif_doublon['Count']=pd.Series(1,range(nrow))  # Créer la colonne 'Count' dans dataframe 'fichier_verif_doublon' et mettre la valeur '1' autant de lignes de valeur de nrow
fichier_verif_doublon_2=fichier_verif_doublon.groupby('NOIDE')['Count'].sum().reset_index()
NOIDE_doublon_3=fichier_verif_doublon_2[fichier_verif_doublon_2['Count']>1]
NOIDE_doublon_3.head()

,NOIDE,Count


In [28]:
NOIDE_doublon_3

,NOIDE,Count


In [19]:
## Liste des doublons à partir un dataframe qui contient la liste des doublons "NOIDE_doublon"
df_Liste_NOIDE_doublon=df_portefeuile_MSP[df_portefeuile_MSP['NOIDE'].isin(list(NOIDE_doublon_3['NOIDE']))].copy() 
df_Liste_NOIDE_doublon=df_Liste_NOIDE_doublon.sort_values(by=['NOIDE'],ascending=False)
df_Liste_NOIDE_doublon.head(60)

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE


In [21]:
## Calculer les effectifs de chaque équipement de fichier "portefeuille MSP" consolidé
liste=['TOP_ASSUR_VIE', 'TOP_DECES', 'TOP_CAM','TOP_TOUT_HABITAT', 'TOP_HABITAT_MGEN', 'TOP_OBS', 'TOP_PIM',
        'TOP_RETRAITE','TOP_PROTECT_JURIQ','TOP_SANTE_ANIM'] 
for elem in liste[:]:
    display(pd.DataFrame(df_portefeuile_MSP[elem].value_counts(sort=True,ascending=True)))

,TOP_ASSUR_VIE
1,4633
0,1850786


,TOP_DECES
1,48341
0,1807078


,TOP_CAM
1,37371
0,1818048


,TOP_TOUT_HABITAT
1,198311
0,1657108


,TOP_HABITAT_MGEN
1,124789
0,1730630


,TOP_OBS
1,74444
0,1780975


,TOP_PIM
1,29003
0,1826416


,TOP_RETRAITE
1,2433
0,1852986


<div class='alert alert-danger'>
    <strong> <h5>Construction de la base consolidée pour l'analyse</h5> </strong>
</div>

<div class='alert alert-info'>
    <strong> <h5>Enrichir les données "Consentement" et "Contact" dans dataframe "df_portefeuile_MSP"</h5> </strong>
</div>

In [22]:
df_base=(df_portefeuile_MSP.merge(df_contact,how='left',on='NOIDE')).merge(df_consentement_mod,how='left',on='NOIDE')
df_base

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE,...,AVEC_ADRESSE,NAT_ACTIVITE,PROMOANACOUR,PROMOANAMAIL,PROMOANASMS,PROMOANATEL,PROMOHANACOUR,PROMOHANAMAIL,PROMOHANASMS,PROMOHANATEL
0,0103544722,1,1992,1,0,0,0,0,0,0,...,1.0,Actif,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0105893027,1,1999,1,0,0,0,0,0,0,...,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0106217145,1,1981,0,0,0,0,0,0,0,...,1.0,Actif,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
3,0107977127,1,1959,0,0,0,0,0,0,0,...,1.0,Actif,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
4,0105891091,1,1997,0,0,0,0,0,0,0,...,1.0,NaN,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855415,0105880759,0,1998,0,0,0,0,0,0,1,...,1.0,NaN,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1855416,0109511742,0,1997,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855417,0104782983,0,1968,0,0,0,0,0,0,1,...,1.0,Actif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1855419 entries, 0 to 1855418
Data columns (total 23 columns):
 #   Column            Dtype  
---  ------            -----  
 0   NOIDE             object 
 1   TOP_ASSUR_VIE     int32  
 2   DATNAI            int64  
 3   TOP_DECES         int32  
 4   TOP_CAM           int32  
 5   TOP_TOUT_HABITAT  int32  
 6   TOP_HABITAT_MGEN  int32  
 7   TOP_OBS           int32  
 8   TOP_PIM           int32  
 9   TOP_RETRAITE      int32  
 10  AVEC_MAIL         float64
 11  AVEC_TEL_FIX      float64
 12  AVEC_TEL_PORT     float64
 13  AVEC_ADRESSE      float64
 14  NAT_ACTIVITE      object 
 15  PROMOANACOUR      float64
 16  PROMOANAMAIL      float64
 17  PROMOANASMS       float64
 18  PROMOANATEL       float64
 19  PROMOHANACOUR     float64
 20  PROMOHANAMAIL     float64
 21  PROMOHANASMS      float64
 22  PROMOHANATEL      float64
dtypes: float64(12), int32(8), int64(1), object(2)
memory usage: 283.1+ MB


In [24]:
## Calculer les effectifs de chaque consentement de dataframe "df_base" consolidé
liste=['PROMOANACOUR', 'PROMOANAMAIL', 'PROMOANASMS','PROMOANATEL', 'PROMOHANACOUR', 'PROMOHANAMAIL', 'PROMOHANASMS',
       'PROMOHANATEL']
for elem in liste[:]:
    display(pd.DataFrame(df_base[elem].value_counts(sort=True,ascending=True)))

,PROMOANACOUR
0.0,42157
1.0,1328908


,PROMOANAMAIL
0.0,94694
1.0,1276371


,PROMOANASMS
0.0,92276
1.0,1278789


,PROMOANATEL
0.0,43668
1.0,1327397


,PROMOHANACOUR
0.0,24531
1.0,1346534


,PROMOHANAMAIL
1.0,20472
0.0,1350593


,PROMOHANASMS
1.0,14136
0.0,1356929


,PROMOHANATEL
0.0,25928
1.0,1345137


In [25]:
# Remplacer les cellules "TRUE" par 1 et "NaN" par 0 dont le libellé des colonnes commence par "PROMO"
colon=[col for col in df_base.columns if col.startswith('PROMO')]

df_base[colon]=df_base[colon].fillna(0)

df_base[colon]=df_base[colon].astype(int)    # Mettre les colonnes concernées en type ENTIER

In [26]:
df_base

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE,...,AVEC_ADRESSE,NAT_ACTIVITE,PROMOANACOUR,PROMOANAMAIL,PROMOANASMS,PROMOANATEL,PROMOHANACOUR,PROMOHANAMAIL,PROMOHANASMS,PROMOHANATEL
0,0103544722,1,1992,1,0,0,0,0,0,0,...,1.0,Actif,1,0,0,1,1,0,0,1
1,0105893027,1,1999,1,0,0,0,0,0,0,...,1.0,NaN,1,1,1,1,1,1,1,1
2,0106217145,1,1981,0,0,0,0,0,0,0,...,1.0,Actif,1,1,1,1,1,0,0,1
3,0107977127,1,1959,0,0,0,0,0,0,0,...,1.0,Actif,1,1,1,1,1,0,0,1
4,0105891091,1,1997,0,0,0,0,0,0,0,...,1.0,NaN,1,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,0,0,0,0,0,0,1,...,NaN,NaN,0,0,0,0,0,0,0,0
1855415,0105880759,0,1998,0,0,0,0,0,0,1,...,1.0,NaN,1,1,1,1,1,0,0,1
1855416,0109511742,0,1997,0,0,0,0,0,0,1,...,NaN,NaN,0,0,0,0,0,0,0,0
1855417,0104782983,0,1968,0,0,0,0,0,0,1,...,1.0,Actif,0,0,0,0,0,0,0,0


In [27]:
## Calculer les effectifs de chaque consentement de dataframe "df_base" consolidé après recodification des colonnes consentement
liste=['PROMOANACOUR', 'PROMOANAMAIL', 'PROMOANASMS','PROMOANATEL', 'PROMOHANACOUR', 'PROMOHANAMAIL', 'PROMOHANASMS',
       'PROMOHANATEL']
for elem in liste[:]:
    display(pd.DataFrame(df_base[elem].value_counts(sort=True,ascending=True)))

,PROMOANACOUR
0,526511
1,1328908


,PROMOANAMAIL
0,579048
1,1276371


,PROMOANASMS
0,576630
1,1278789


,PROMOANATEL
0,528022
1,1327397


,PROMOHANACOUR
0,508885
1,1346534


,PROMOHANAMAIL
1,20472
0,1834947


,PROMOHANASMS
1,14136
0,1841283


,PROMOHANATEL
0,510282
1,1345137


In [29]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1903651 entries, 0 to 1903650
Data columns (total 25 columns):
 #   Column             Dtype  
---  ------             -----  
 0   NOIDE              object 
 1   TOP_ASSUR_VIE      int32  
 2   DATNAI             int64  
 3   TOP_DECES          int32  
 4   TOP_CAM            int32  
 5   TOP_TOUT_HABITAT   int32  
 6   TOP_HABITAT_MGEN   int32  
 7   TOP_OBS            int32  
 8   TOP_PIM            int32  
 9   TOP_PROTECT_JURIQ  int32  
 10  TOP_RETRAITE       int32  
 11  TOP_SANTE_ANIM     int32  
 12  AVEC_MAIL          float64
 13  AVEC_TEL_FIX       float64
 14  AVEC_TEL_PORT      float64
 15  AVEC_ADRESSE       float64
 16  NAT_ACTIVITE       object 
 17  PROMOANACOUR       int32  
 18  PROMOANAMAIL       int32  
 19  PROMOANASMS        int32  
 20  PROMOANATEL        int32  
 21  PROMOHANACOUR      int32  
 22  PROMOHANAMAIL      int32  
 23  PROMOHANASMS       int32  
 24  PROMOHANATEL       int32  
dtypes: float64(4), int

<div class='alert alert-danger'>
    <strong> <h5>Création des flags de consentements des adhérents non équipés par offre et par type de canal</h5> </strong>
</div>

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "PIM" par type de canal</h5> </strong>
</div>

In [31]:
# Flag "PIM courriel"
df_base.loc[(df_base['TOP_PIM']==0)&(df_base['PROMOHANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"PIM_Cour"]=1
df_base["PIM_Cour"].fillna(0,inplace=True)

# Flag "PIM Mail"
df_base.loc[(df_base['TOP_PIM']==0)&(df_base['PROMOHANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"PIM_MAIL"]=1
df_base["PIM_MAIL"].fillna(0,inplace=True)

# Flag "PIM SMS"
df_base.loc[(df_base['TOP_PIM']==0)&(df_base['PROMOHANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"PIM_SMS"]=1
df_base["PIM_SMS"].fillna(0,inplace=True)

# Flag "PIM TEL"
df_base.loc[(df_base['TOP_PIM']==0)&(df_base['PROMOHANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"PIM_TEL"]=1
df_base["PIM_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "RETRAITE" par type de canal</h5> </strong>
</div>

In [32]:
# RETRAITE courriel
df_base.loc[(df_base['TOP_RETRAITE']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"RETRAITE_Cour"]=1
df_base["RETRAITE_Cour"].fillna(0,inplace=True)

# RETRAITE Mail
df_base.loc[(df_base['TOP_RETRAITE']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"RETRAITE_MAIL"]=1
df_base["RETRAITE_MAIL"].fillna(0,inplace=True)

# RETRAITE SMS
df_base.loc[(df_base['TOP_RETRAITE']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"RETRAITE_SMS"]=1
df_base["RETRAITE_SMS"].fillna(0,inplace=True)

# RETRAITE TEL
df_base.loc[(df_base['TOP_RETRAITE']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"RETRAITE_TEL"]=1
df_base["RETRAITE_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "AUSSRANCE-VIE" par type de canal</h5> </strong>
</div>

In [33]:
# Assur_Vie courriel
df_base.loc[(df_base['TOP_ASSUR_VIE']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"ASSUR_VIE_Cour"]=1
df_base["ASSUR_VIE_Cour"].fillna(0,inplace=True)

# Assur_Vie Mail
df_base.loc[(df_base['TOP_ASSUR_VIE']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"ASSUR_VIE_MAIL"]=1
df_base["ASSUR_VIE_MAIL"].fillna(0,inplace=True)

# Assur_Vie SMS
df_base.loc[(df_base['TOP_ASSUR_VIE']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"ASSUR_VIE_SMS"]=1
df_base["ASSUR_VIE_SMS"].fillna(0,inplace=True)

# Assur_Vie TEL
df_base.loc[(df_base['TOP_ASSUR_VIE']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"ASSUR_VIE_TEL"]=1
df_base["ASSUR_VIE_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "AUSSRANCE-DECES" par type de canal</h5> </strong>
</div>

In [34]:
# DECES courriel
df_base.loc[(df_base['TOP_DECES']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"DECES_Cour"]=1
df_base["DECES_Cour"].fillna(0,inplace=True)

# DECES Mail
df_base.loc[(df_base['TOP_DECES']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"DECES_MAIL"]=1
df_base["DECES_MAIL"].fillna(0,inplace=True)

# DECES SMS
df_base.loc[(df_base['TOP_DECES']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"DECES_SMS"]=1
df_base["DECES_SMS"].fillna(0,inplace=True)

# DECES TEL
df_base.loc[(df_base['TOP_DECES']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"DECES_TEL"]=1
df_base["DECES_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "AUSSRANCE-OBSEQUES" par type de canal</h5> </strong>
</div>

In [35]:
# Obseques courriel
df_base.loc[(df_base['TOP_OBS']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"OBS_Cour"]=1
df_base["OBS_Cour"].fillna(0,inplace=True)

# Obseques Mail
df_base.loc[(df_base['TOP_OBS']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"OBS_MAIL"]=1
df_base["OBS_MAIL"].fillna(0,inplace=True)

# Obseques SMS
df_base.loc[(df_base['TOP_OBS']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"OBS_SMS"]=1
df_base["OBS_SMS"].fillna(0,inplace=True)

# Obseques TEL
df_base.loc[(df_base['TOP_OBS']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"OBS_TEL"]=1
df_base["OBS_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "CAM" par type de canal</h5> </strong>
</div>

In [36]:
# CAM courriel
df_base.loc[(df_base['TOP_CAM']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"CAM_Cour"]=1
df_base["CAM_Cour"].fillna(0,inplace=True)

# CAM Mail
df_base.loc[(df_base['TOP_CAM']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"CAM_MAIL"]=1
df_base["CAM_MAIL"].fillna(0,inplace=True)

# CAM SMS
df_base.loc[(df_base['TOP_CAM']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"CAM_SMS"]=1
df_base["CAM_SMS"].fillna(0,inplace=True)

# CAM TEL
df_base.loc[(df_base['TOP_CAM']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"CAM_TEL"]=1
df_base["CAM_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "TOUT_HABITAT" par type de canal</h5> </strong>
</div>

In [37]:
# TOUT_HABITAT courriel
df_base.loc[(df_base['TOP_TOUT_HABITAT']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"TOUT_HABITAT_Cour"]=1
df_base["TOUT_HABITAT_Cour"].fillna(0,inplace=True)

# TOUT_HABITAT Mail
df_base.loc[(df_base['TOP_TOUT_HABITAT']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"TOUT_HABITAT_MAIL"]=1
df_base["TOUT_HABITAT_MAIL"].fillna(0,inplace=True)

# TOUT_HABITAT SMS
df_base.loc[(df_base['TOP_TOUT_HABITAT']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"TOUT_HABITAT_SMS"]=1
df_base["TOUT_HABITAT_SMS"].fillna(0,inplace=True)

# TOUT_HABITAT TEL
df_base.loc[(df_base['TOP_TOUT_HABITAT']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"TOUT_HABITAT_TEL"]=1
df_base["TOUT_HABITAT_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "HABITAT MGEN" par type de canal</h5> </strong>
</div>

In [38]:
## 1--> c'est un potentiel via le canal 
## 0--> ce n'est pas un potentiel

# HABITAT_MGEN courriel
df_base.loc[(df_base['TOP_HABITAT_MGEN']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"HABITAT_MGEN_Cour"]=1
df_base["HABITAT_MGEN_Cour"].fillna(0,inplace=True)

# HABITAT_MGEN Mail
df_base.loc[(df_base['TOP_HABITAT_MGEN']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"HABITAT_MGEN_MAIL"]=1
df_base["HABITAT_MGEN_MAIL"].fillna(0,inplace=True)

# HABITAT_MGEN SMS
df_base.loc[(df_base['TOP_HABITAT_MGEN']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"HABITAT_MGEN_SMS"]=1
df_base["HABITAT_MGEN_SMS"].fillna(0,inplace=True)

# HABITAT_MGEN TEL
df_base.loc[(df_base['TOP_HABITAT_MGEN']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"HABITAT_MGEN_TEL"]=1
df_base["HABITAT_MGEN_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "SANTE ANIMALE" par type de canal</h5> </strong>
</div>

In [ ]:
# Santé animale courriel
df_base.loc[(df_base['TOP_SANTE_ANIM']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"SANTE_ANIM_Cour"]=1
df_base["SANTE_ANIM_Cour"].fillna(0,inplace=True)

# Santé animale Mail
df_base.loc[(df_base['TOP_SANTE_ANIM']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"SANTE_ANIM_MAIL"]=1
df_base["SANTE_ANIM_MAIL"].fillna(0,inplace=True)

# Santé animale SMS
df_base.loc[(df_base['TOP_SANTE_ANIM']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"SANTE_ANIM_SMS"]=1
df_base["SANTE_ANIM_SMS"].fillna(0,inplace=True)

# Santé animale TEL
df_base.loc[(df_base['TOP_SANTE_ANIM']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"SANTE_ANIM_TEL"]=1
df_base["SANTE_ANIM_TEL"].fillna(0,inplace=True)

<div class='alert alert-info'>
    <strong> <h5>Création flags de consentements des adhérents non équipé "PROTECTION JURIDIQUE" par type de canal</h5> </strong>
</div>

In [ ]:
# PROTECT_JURIQ courriel
df_base.loc[(df_base['TOP_PROTECT_JURIQ']==0)&(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"PROTECT_JURIQ_Cour"]=1
df_base["PROTECT_JURIQ_Cour"].fillna(0,inplace=True)

# PROTECT_JURIQ Mail
df_base.loc[(df_base['TOP_PROTECT_JURIQ']==0)&(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"PROTECT_JURIQ_MAIL"]=1
df_base["PROTECT_JURIQ_MAIL"].fillna(0,inplace=True)

# PROTECT_JURIQ SMS
df_base.loc[(df_base['TOP_PROTECT_JURIQ']==0)&(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"PROTECT_JURIQ_SMS"]=1
df_base["PROTECT_JURIQ_SMS"].fillna(0,inplace=True)

# PROTECT_JURIQ TEL
df_base.loc[(df_base['TOP_PROTECT_JURIQ']==0)&(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"PROTECT_JURIQ_TEL"]=1
df_base["PROTECT_JURIQ_TEL"].fillna(0,inplace=True)

<div class='alert alert-danger'>
    <strong> <h5>Création des flags de consentements par type de canal</h5> </strong>
</div>

## Ne pas executer car n'est pas utilisé ici

In [40]:
## Consentement AVEC H

# Flag "consentement courriel avec H"
df_base.loc[(df_base['PROMOHANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"Consent_Cour_avec_H"]=1
df_base["Consent_Cour_avec_H"].fillna(0,inplace=True)

# Flag "consentement Mail avec H"
df_base.loc[(df_base['PROMOHANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"Consent_Mail_avec_H"]=1
df_base["Consent_Mail_avec_H"].fillna(0,inplace=True)

# Flag "consentement SMS avec H"
df_base.loc[(df_base['PROMOHANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"Consent_SMS_avec_H"]=1
df_base["Consent_SMS_avec_H"].fillna(0,inplace=True)

# Flag "consentement TEL avec H"
df_base.loc[(df_base['PROMOHANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"Consent_Tel_avec_H"]=1
df_base["Consent_Tel_avec_H"].fillna(0,inplace=True)



## Consentement SANS H
# Flag "consentement courriel"
df_base.loc[(df_base['PROMOANACOUR']==1)&(df_base['AVEC_ADRESSE']==1),"Consent_Cour"]=1
df_base["Consent_Cour"].fillna(0,inplace=True)

# Flag "consentement Mail"
df_base.loc[(df_base['PROMOANAMAIL']==1)&(df_base['AVEC_MAIL']==1),"Consent_Mail"]=1
df_base["Consent_Mail"].fillna(0,inplace=True)

# Flag "consentement SMS"
df_base.loc[(df_base['PROMOANASMS']==1)&(df_base['AVEC_TEL_PORT']==1),"Consent_SMS"]=1
df_base["Consent_SMS"].fillna(0,inplace=True)

# Flag "consentement TEL"
df_base.loc[(df_base['PROMOANATEL']==1)&((df_base['AVEC_TEL_PORT']==1)|(df_base['AVEC_TEL_FIX']==1)),"Consent_Tel"]=1
df_base["Consent_Tel"].fillna(0,inplace=True)

## Continuer l'execution

In [39]:
df_base

,NOIDE,TOP_ASSUR_VIE,DATNAI,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,TOP_RETRAITE,...,CAM_SMS,CAM_TEL,TOUT_HABITAT_Cour,TOUT_HABITAT_MAIL,TOUT_HABITAT_SMS,TOUT_HABITAT_TEL,HABITAT_MGEN_Cour,HABITAT_MGEN_MAIL,HABITAT_MGEN_SMS,HABITAT_MGEN_TEL
0,0103544722,1,1992,1,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0105893027,1,1999,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0106217145,1,1981,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0107977127,1,1959,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0105891091,1,1997,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855415,0105880759,0,1998,0,0,0,0,0,0,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1855416,0109511742,0,1997,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855417,0104782983,0,1968,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<div class='alert alert-danger'>
    <strong> <h5>Création de colonne AGE</h5> </strong>
</div>

In [40]:
df_base.insert(3,"Age",2022-df_base['DATNAI']) # 3 pour dire qu'on veut que la colonne AGE soit en colonne 4 du Dataframe


In [41]:
df_base

,NOIDE,TOP_ASSUR_VIE,DATNAI,Age,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,...,CAM_SMS,CAM_TEL,TOUT_HABITAT_Cour,TOUT_HABITAT_MAIL,TOUT_HABITAT_SMS,TOUT_HABITAT_TEL,HABITAT_MGEN_Cour,HABITAT_MGEN_MAIL,HABITAT_MGEN_SMS,HABITAT_MGEN_TEL
0,0103544722,1,1992,30,1,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0105893027,1,1999,23,1,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0106217145,1,1981,41,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0107977127,1,1959,63,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0105891091,1,1997,25,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855415,0105880759,0,1998,24,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1855416,0109511742,0,1997,25,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855417,0104782983,0,1968,54,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<div class='alert alert-danger'>
    <strong> <h5>Chargement du fichier des devis</h5> </strong>
</div>

In [46]:
# Import fichier "Proposition" de 2020-01-01 au 2022-12-31
## Modifier en fonction à partir de trois ans à compter de la date d'extraction

df_devis=pd.read_csv(r"P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Input_Files\NOIDE_DEVIS.csv",
                           delimiter="|", engine="python", dtype={"NOIDE":str,"DATCRE":datetime.datetime}, encoding="mbcs")

df_devis["DATCRE"]=pd.to_datetime(df_devis["DATCRE"])

In [47]:
## Dédoublonnage du fichier de proposition en prenant la proposition la plus recente

df_devis=df_devis.sort_values("DATCRE",ascending=False)

## On supprime les heure/min/s de la colonne date

df_devis["DATCRE"]=df_devis["DATCRE"].apply(lambda x: x.date())

## On prend la proposition la plus recente

df_devis.drop_duplicates(keep='first',inplace=True)

df_devis=df_devis.rename(columns={"CODCNLPRP":"CANAL_PROP"})

In [51]:
## Vérification d'absence de doublon

df_double=df_devis.copy()

df_double["Count"]=1

date_test=df_double.groupby(["NOIDE","EQUIPEMENT","DATCRE","CANAL_PROP"]).sum()

date_test_double=date_test[date_test["Count"]>1]

date_test_double.groupby("EQUIPEMENT").count()

,Count
EQUIPEMENT,


In [52]:
df_double[df_double.duplicated()]

,NOIDE,EQUIPEMENT,DATCRE,CANAL_PROP,Count


In [53]:
## Croissement entre les noides les devis pour avoir pour chaque noide le nombre de devis associé à un équipement.

devis_date=pd.pivot_table(df_devis[["NOIDE","EQUIPEMENT","DATCRE"]], index=['NOIDE'],columns=['EQUIPEMENT'],aggfunc='count',fill_value=0) 

devis_date

DATCRE                                            \
EQUIPEMENT Devis_Assur_Décès Devis_Assur_Obs Devis_Assur_Vie Devis_CAM   
NOIDE                                                                    
0100000001                 0               0               0         1   
0100009089                 0               1               0         0   
0100011422                 0               0               0         0   
0100013758                 1               0               0         1   
0100015658                 0               1               0         0   
...                      ...             ...             ...       ...   
0109547155                 0               0               1         0   
0109547613                 0               0               1         0   
0109548509                 0               0               0         0   
0109548575                 1               0               0         0   
0109548643                 1               0               0         0   

                                                                 \
EQUIPEMENT Devis_Epargne_retraite Devis_PIM Devis_Protect_Juriq   
NOIDE                                                             
0100000001                      0         0                   0   
0100009089                      0         0                   0   
0100011422                      1         0                   0   
0100013758                      0         0                   0   
0100015658                      0         0                   0   
...                           ...       ...                 ...   
0109547155                      0         0                   0   
0109547613                      0         0                   0   
0109548509                      0         1                   0   
0109548575                      0         0                   0   
0109548643                      0         0                   0   

                                
EQUIPEMENT Devis_Sante_Animale  
NOIDE                           
0100000001                   0  
0100009089                   0  
0100011422                   0  
0100013758                   0  
0100015658                   0  
...                        ...  
0109547155                   0  
0109547613                   0  
0109548509                   0  
0109548575                   0  
0109548643                   0  

[191851 rows x 8 columns]

In [54]:
## Transformation du multiindex en dataframe

devis_date.columns=devis_date.columns.droplevel(0)

devis_date.columns.name=None

devis_date=devis_date.reset_index()

devis_date

,NOIDE,Devis_Assur_Décès,Devis_Assur_Obs,Devis_Assur_Vie,Devis_CAM,Devis_Epargne_retraite,Devis_PIM,Devis_Protect_Juriq,Devis_Sante_Animale
0,0100000001,0,0,0,1,0,0,0,0
1,0100009089,0,1,0,0,0,0,0,0
2,0100011422,0,0,0,0,1,0,0,0
3,0100013758,1,0,0,1,0,0,0,0
4,0100015658,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
191846,0109547155,0,0,1,0,0,0,0,0
191847,0109547613,0,0,1,0,0,0,0,0
191848,0109548509,0,0,0,0,0,1,0,0
191849,0109548575,1,0,0,0,0,0,0,0


In [56]:
## Jointure avec la base initiale

df_base_devis=df_base.merge(devis_date,how='left',on='NOIDE')

df_base_devis

,NOIDE,TOP_ASSUR_VIE,DATNAI,Age,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,...,HABITAT_MGEN_Cour,HABITAT_MGEN_MAIL,HABITAT_MGEN_SMS,HABITAT_MGEN_TEL,Devis_Assur_Décès,Devis_Assur_Obs,Devis_Assur_Vie,Devis_CAM,Devis_Epargne_retraite,Devis_PIM
0,0103544722,1,1992,30,1,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0105893027,1,1999,23,1,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0106217145,1,1981,41,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0107977127,1,1959,63,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0105891091,1,1997,25,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1855415,0105880759,0,1998,24,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1855416,0109511742,0,1997,25,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1855417,0104782983,0,1968,54,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


## Ne pas executer car n'est pas utilisé ici

In [62]:
## Croissement entre les noides les devis pour avoir pour chaque noide la date de devis la plus récente associée à un équipement.

devis_date_date=pd.pivot_table(df_devis[["NOIDE","EQUIPEMENT","DATCRE"]], index=['NOIDE'],columns=['EQUIPEMENT'],aggfunc='max') 

devis_date_date

DATCRE                                              \
EQUIPEMENT Devis_Assur_Décès Devis_Assur_Obs Devis_Assur_Vie   Devis_CAM   
NOIDE                                                                      
0100000001               NaN             NaN             NaN  2022-02-18   
0100009089               NaN      2021-10-22             NaN         NaN   
0100011422               NaN             NaN             NaN         NaN   
0100013758        2022-04-04             NaN             NaN  2022-04-28   
0100015658               NaN      2022-08-23             NaN         NaN   
...                      ...             ...             ...         ...   
0109517942        2022-02-09             NaN             NaN         NaN   
0109518030        2022-12-20             NaN             NaN         NaN   
0109518101               NaN             NaN             NaN         NaN   
0109518824        2022-09-06             NaN      2022-09-06         NaN   
Z#                2021-12-20             NaN             NaN  2021-12-13   

                                                                   \
EQUIPEMENT Devis_Epargne_retraite   Devis_PIM Devis_Protect_Juriq   
NOIDE                                                               
0100000001                    NaN         NaN                 NaN   
0100009089                    NaN         NaN                 NaN   
0100011422             2022-05-18         NaN                 NaN   
0100013758                    NaN         NaN                 NaN   
0100015658                    NaN         NaN                 NaN   
...                           ...         ...                 ...   
0109517942                    NaN         NaN                 NaN   
0109518030             2022-12-20         NaN          2022-12-20   
0109518101                    NaN  2021-09-23                 NaN   
0109518824                    NaN         NaN                 NaN   
Z#                            NaN         NaN                 NaN   

                                
EQUIPEMENT Devis_Sante_Animale  
NOIDE                           
0100000001                 NaN  
0100009089                 NaN  
0100011422                 NaN  
0100013758                 NaN  
0100015658                 NaN  
...                        ...  
0109517942                 NaN  
0109518030                 NaN  
0109518101                 NaN  
0109518824                 NaN  
Z#                         NaN  

[258944 rows x 8 columns]

In [63]:
## Transformation du multiindex en dataframe

devis_date_date.columns=devis_date_date.columns.droplevel(0)

devis_date_date.columns.name=None

devis_date_date.columns=['Date_'+columns for columns in devis_date_date.columns]

devis_date_date=devis_date_date.reset_index()

devis_date_date

,NOIDE,Date_Devis_Assur_Décès,Date_Devis_Assur_Obs,Date_Devis_Assur_Vie,Date_Devis_CAM,Date_Devis_Epargne_retraite,Date_Devis_PIM,Date_Devis_Protect_Juriq,Date_Devis_Sante_Animale
0,0100000001,NaN,NaN,NaN,2022-02-18,NaN,NaN,NaN,NaN
1,0100009089,NaN,2021-10-22,NaN,NaN,NaN,NaN,NaN,NaN
2,0100011422,NaN,NaN,NaN,NaN,2022-05-18,NaN,NaN,NaN
3,0100013758,2022-04-04,NaN,NaN,2022-04-28,NaN,NaN,NaN,NaN
4,0100015658,NaN,2022-08-23,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
258939,0109517942,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258940,0109518030,2022-12-20,NaN,NaN,NaN,2022-12-20,NaN,2022-12-20,NaN
258941,0109518101,NaN,NaN,NaN,NaN,NaN,2021-09-23,NaN,NaN
258942,0109518824,2022-09-06,NaN,2022-09-06,NaN,NaN,NaN,NaN,NaN


In [64]:
## Jointure avec la base initiale

df_base_devis=df_base_devis.merge(devis_date_date,how='left',on='NOIDE')

df_base_devis

,NOIDE,TOP_ASSUR_VIE,DATNAI,Age,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,...,Devis_Protect_Juriq,Devis_Sante_Animale,Date_Devis_Assur_Décès,Date_Devis_Assur_Obs,Date_Devis_Assur_Vie,Date_Devis_CAM,Date_Devis_Epargne_retraite,Date_Devis_PIM,Date_Devis_Protect_Juriq,Date_Devis_Sante_Animale
0,0105762241,1,2002,20,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,2021-12-15,NaN,NaN,NaN,NaN,NaN
1,0105709998,1,1995,27,0,0,0,0,0,0,...,0.0,0.0,2021-08-28,NaN,2021-08-28,NaN,NaN,NaN,NaN,NaN
2,0105537911,1,1973,49,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,2022-07-22,NaN,NaN,NaN,NaN,NaN
3,0107155834,1,1963,59,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,2022-07-11,2022-02-01,NaN,NaN,NaN,NaN
4,0100611640,1,1963,59,0,0,0,0,1,0,...,0.0,0.0,NaN,NaN,2020-12-16,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898449,0105261161,0,1969,53,0,0,0,0,0,0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-10,2022-11-28
1898450,0107771234,0,1995,27,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-15
1898451,0101536995,0,1945,77,0,0,0,0,0,0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-04,2022-10-24
1898452,0108774779,0,1973,49,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-02


## Continuer l'execution

In [57]:
## Croissement pour avoir chaque noide le canal de la proposition

df_devis["CANAL_PROPO"]=df_devis["CANAL_PROP"].map({"CORRESPONDANCE":"Courrier","COURRIEL":"Mail","FACEAFACE":"FACEAFACE",
                          "INTERNET":"INTERNET","TELEPHONE":"TELEPHONE"},na_action='ignore')

devis_canal=pd.pivot_table(df_devis[["NOIDE","EQUIPEMENT","CANAL_PROPO"]], index=['NOIDE'],columns=['CANAL_PROPO'],aggfunc="count",fill_value=0) 

## Transformation du multiindex en dataframe

devis_canal.columns=devis_canal.columns.droplevel(0)

devis_canal.columns.name=None

devis_canal.columns=['PROPO_'+columns for columns in devis_canal.columns]

devis_canal=devis_canal.reset_index()

devis_canal

## Propo_courrier=1 ==> le noide a fait une proposition via le canal Courrier entre 2020-01-01 et 2022-12-31
## Propo_FACEAFACE ==> idem

,NOIDE,PROPO_Courrier,PROPO_FACEAFACE,PROPO_INTERNET,PROPO_Mail,PROPO_TELEPHONE
0,0100000001,0,1,0,0,0
1,0100009089,0,1,0,0,0
2,0100011422,0,1,0,0,0
3,0100013758,0,2,0,0,0
4,0100015658,0,1,0,0,0
...,...,...,...,...,...,...
191846,0109547155,0,0,0,0,1
191847,0109547613,0,1,0,0,0
191848,0109548509,0,0,0,0,1
191849,0109548575,0,1,0,0,0


In [58]:
## Jointure avec la base initiale

df_base_devis=df_base_devis.merge(devis_canal,how='left',on='NOIDE')

df_base_devis

,NOIDE,TOP_ASSUR_VIE,DATNAI,Age,TOP_DECES,TOP_CAM,TOP_TOUT_HABITAT,TOP_HABITAT_MGEN,TOP_OBS,TOP_PIM,...,Devis_Assur_Obs,Devis_Assur_Vie,Devis_CAM,Devis_Epargne_retraite,Devis_PIM,PROPO_Courrier,PROPO_FACEAFACE,PROPO_INTERNET,PROPO_Mail,PROPO_TELEPHONE
0,0103544722,1,1992,30,1,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0105893027,1,1999,23,1,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
2,0106217145,1,1981,41,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0107977127,1,1959,63,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0105891091,1,1997,25,0,0,0,0,0,0,...,0.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855414,0109509721,0,1994,28,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855415,0105880759,0,1998,24,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1855416,0109511742,0,1997,25,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855417,0104782983,0,1968,54,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
devis_date[devis_date.columns[1:]].max()

Devis_Assur_Décès         18
Devis_Assur_Obs            5
Devis_Assur_Vie            8
Devis_CAM                  6
Devis_Epargne_retraite     6
Devis_PIM                  4
Devis_Protect_Juriq        4
Devis_Sante_Animale        6
dtype: int64

In [65]:
## Maximum de devis par équipement

devis_date[devis_date.columns[1:]].max()

Devis_Assur_Décès         16
Devis_Assur_Obs            5
Devis_Assur_Vie            8
Devis_CAM                  6
Devis_Epargne_retraite     6
Devis_PIM                  4
Devis_Protect_Juriq        4
Devis_Sante_Animale        5
dtype: int64

In [66]:
devis_date[devis_date["Devis_Assur_Décès"]==16]

,NOIDE,Devis_Assur_Décès,Devis_Assur_Obs,Devis_Assur_Vie,Devis_CAM,Devis_Epargne_retraite,Devis_PIM,Devis_Protect_Juriq,Devis_Sante_Animale
243633,0108658527,16,3,3,1,1,0,1,0


In [64]:
df_devis[df_devis["NOIDE"]=="0107229696"]

,NOIDE,EQUIPEMENT,DATCRE,CANAL_PROP,CANAL_PROPO
296887,0107229696,Devis_Assur_Décès,2022-11-10,TELEPHONE,TELEPHONE
251031,0107229696,Devis_Protect_Juriq,2022-11-08,TELEPHONE,TELEPHONE
234973,0107229696,Devis_Assur_Décès,2022-10-14,TELEPHONE,TELEPHONE
61618,0107229696,Devis_Assur_Décès,2022-10-11,TELEPHONE,TELEPHONE
270591,0107229696,Devis_Protect_Juriq,2022-10-10,TELEPHONE,TELEPHONE
265620,0107229696,Devis_Assur_Décès,2022-10-05,TELEPHONE,TELEPHONE
300631,0107229696,Devis_Assur_Décès,2022-06-02,TELEPHONE,TELEPHONE
319386,0107229696,Devis_Assur_Décès,2022-05-24,TELEPHONE,TELEPHONE
325974,0107229696,Devis_Assur_Décès,2022-05-10,COURRIEL,Mail
338574,0107229696,Devis_Assur_Décès,2022-05-10,TELEPHONE,TELEPHONE


In [67]:
df_base_devis.columns

Index(['NOIDE', 'TOP_ASSUR_VIE', 'DATNAI', 'Age', 'TOP_DECES', 'TOP_CAM',
       'TOP_TOUT_HABITAT', 'TOP_HABITAT_MGEN', 'TOP_OBS', 'TOP_PIM',
       'TOP_PROTECT_JURIQ', 'TOP_RETRAITE', 'TOP_SANTE_ANIM', 'AVEC_MAIL',
       'AVEC_TEL_FIX', 'AVEC_TEL_PORT', 'AVEC_ADRESSE', 'NAT_ACTIVITE',
       'PROMOANACOUR', 'PROMOANAMAIL', 'PROMOANASMS', 'PROMOANATEL',
       'PROMOHANACOUR', 'PROMOHANAMAIL', 'PROMOHANASMS', 'PROMOHANATEL',
       'PIM_Cour', 'PIM_MAIL', 'PIM_SMS', 'PIM_TEL', 'RETRAITE_Cour',
       'RETRAITE_MAIL', 'RETRAITE_SMS', 'RETRAITE_TEL', 'ASSUR_VIE_Cour',
       'ASSUR_VIE_MAIL', 'ASSUR_VIE_SMS', 'ASSUR_VIE_TEL', 'DECES_Cour',
       'DECES_MAIL', 'DECES_SMS', 'DECES_TEL', 'OBS_Cour', 'OBS_MAIL',
       'OBS_SMS', 'OBS_TEL', 'CAM_Cour', 'CAM_MAIL', 'CAM_SMS', 'CAM_TEL',
       'TOUT_HABITAT_Cour', 'TOUT_HABITAT_MAIL', 'TOUT_HABITAT_SMS',
       'TOUT_HABITAT_TEL', 'HABITAT_MGEN_Cour', 'HABITAT_MGEN_MAIL',
       'HABITAT_MGEN_SMS', 'HABITAT_MGEN_TEL', 'SANTE_ANIM_Cour',
 

In [59]:
df_base_devis.to_pickle("P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Projet_SCORING_Equip\Input_Files_2023\df_base_devis_2023")

# FIN

<div class='alert alert-danger'>
    <strong> <h5>Sortir les stats de potentiels par offre et par canal: Automatisation faite pour une demande le 10/12/2022</h5> </strong>
</div>

In [129]:
## Flag Activite
## Autres peut contenir les valeurs manquantes

df_base.loc[df_base["NAT_ACTIVITE"]=="Actif","Activite"]="Actif"

df_base.loc[df_base["NAT_ACTIVITE"]=="Retraité","Activite"]="Retraité"

df_base["Activite"].fillna("Autres",inplace=True)

In [130]:
df_base["NAT_ACTIVITE"].value_counts(dropna=False)

Actif                 887622
Retraité              874242
NaN                   102629
Scolarité              21409
Sans activité          16785
TNS et Agricole          251
Service Volontaire         3
Name: NAT_ACTIVITE, dtype: int64

In [352]:
## On définir une fonction qui aura pour but de générer automatiquement le tableau de stat des potentiel.
## Cette fonction prendra au moins 03 paramètres:
## l'equipement déténu (Equip): PIM, RETRAITE ........., écrit exactement comme défini dans les données précédents
## Tranche d'age ('[a,b]'==> a<= x <=b ou '[a,b['==> a<= x <b , ou '[]'==> pas de filtration sur l'age ) 
## Mail_Tel= False par defaut, à modifier à True si on veut les potentiels qui ont ces deux canaux
## SMS_TEL idem
## SMS_MAIL  idem
## MAIL_SMS_TEL  idem
##df ==> dataframe sur lequel seront appliqué les stats
## tranche_age='[]', Mail_Tel=False, SMS_TEL=False, SMS_MAIL=False, MAIL_SMS_TEL=False ==> les critères par défaut au début
## pour tranche age = <70 entrer '[0,70['

##**kwargs==> les paramètres seront entrés sous forme clé=valeur (exemple: SMS_TEL(clé)=False/True(valeur))

## *args==> les paramètres seront entrés sous forme de valeur (exemple: df, Equip)


def Stat_auto(df,Equip,Tranche_age='[]',**kwargs): 
    
    ## **kwargs==> utiliser lorsqu'on ne connait le nombre de paramètre exacte à appliquer au départ.
    
    def Stat_global(df,Equip,kwargs): ## fonction pour la sortie des stats par canal
        
        ## le bloc suivant permet de gérer le mono canal
    
        all_channel=["Cour","MAIL","SMS","TEL"]

        df_chanel=df[df[Equip+"_Cour"]==1].groupby("Activite").sum()[Equip+"_Cour"].to_frame()

        for channel in all_channel[1:]: ##==> à partir de MAIL dans la liste all_channel

            df1=df[df[Equip+"_"+channel]==1].groupby("Activite").sum()[Equip+"_"+channel].to_frame()

            df_chanel=df_chanel.merge(df1, how="outer", left_index=True, right_index=True) ## ==> index=Activité soit actif, retraité ou autres
        
        ## le bloc suivant pour gérer le multicanal
        
        compteur=0

        for channel, val in kwargs.items():  ## kwargs.items()==> récupère tous les couple clé valeur(exemple: (SMS_TEL,True/False))

            if val==True:

                df_equip=(df[(df[Equip+"_"+channel.split("_")[0]]==1) & (df[Equip+"_"+channel.split("_")[1]]==1)] if len(channel.split("_"))==2 else

                          df[(df[Equip+"_"+channel.split("_")[0]]==1) & (df[Equip+"_"+channel.split("_")[1]]==1) & (df[Equip+"_"+channel.split("_")[1]]==1)])

                df_all=(df_equip.groupby("Activite").sum()[Equip+"_"+channel.split("_")[0]].to_frame().rename(columns={Equip+"_"+channel.split("_")[0]:Equip+"_"+channel}) if compteur==0 else
                       
                       df_all.merge(df_equip.groupby("Activite").sum()[Equip+"_"+channel.split("_")[0]].to_frame().rename(columns={Equip+"_"+channel.split("_")[0]:Equip+"_"+channel}),how="outer", left_index=True, right_index=True ))
                
                compteur=1
                
                ## La liste suivante c'est merger les dataframes de mono et multicanal
        
        df_final=df_chanel.merge(df_all,how="outer", left_index=True, right_index=True) if list(kwargs.values()).count(True)>=1 else df_chanel
        
        return df_final   ## sortie des stats
    
    def Stat_age(df,Equip,Tranche_age,kwargs): ## fonction pour filtration du dataframe avec l'age et sortie des stats
    
        if Tranche_age=='[]':

            df_equip= df.copy()

        elif Tranche_age[-1]=='[': ## Vérifie si la borne supérieur de la variable tranche_age est ouverte

            int_age= re.findall("\d+",Tranche_age)  ## pour [18 ans, 74 ans[, int_age sera [18,74]

            df_equip= df[(df["Age"]>= int(int_age[0])) & (df["Age"]<int(int_age[1]))].copy()

        else:

            int_age= re.findall("\d+",Tranche_age)

            df_equip= df[(df["Age"]>= int(int_age[0])) & (df["Age"]<=int(int_age[1]))].copy()
            
            
        return  Stat_global(df_equip,Equip,kwargs)  ## appel de la première fonction pour la sortie des stats en fonction de l'age
    
    
    return (Stat_global(df,Equip,kwargs), Stat_age(df,Equip,Tranche_age,kwargs))

In [353]:
#Stat_auto(df=df_base,Equip="PIM",Tranche_age="[18 ans,36 ans]",MAIL_TEL=False,MAIL_SMS=False)

In [357]:
## aggrégation des données
## les paramètres à paramétrer pour la fonction ci-dessus
## Attention !!! bien écrire les bornes supérieures de la variable tranche d'age

liste_equipement=[("PIM","[18 ans,36 ans]"),("RETRAITE","[18 ans,74 ans]") ,("ASSUR_VIE","[]"),("DECES","[25 ans, 55ans]"),
                 
                 ("OBS","[55 ans, 84 ans["), ("CAM","[55 ans, 74 ans["), ("TOUT_HABITAT","[0,70 ans["),("SANTE_ANIM","[]"),("PROTECT_JURIQ","[]")]
liste_df=[]

for equip in liste_equipement:
    
    global_stat, age_stat= Stat_auto(df=df_base,Equip=equip[0],Tranche_age=equip[1],TEL_SMS=False,TEL_MAIL=True) ## modifier pour avoir le canal qu'on veut
    
    liste1=["Total"]+list(global_stat.sum(axis=0))
    
    liste2=["Total"]+list(age_stat.sum(axis=0))

    global_stat,age_stat=global_stat.reset_index(),age_stat.reset_index()
    
    global_stat.loc[3]=liste1
    
    age_stat.loc[3]=liste2
    
    global_stat,age_stat=global_stat.set_index("Activite").unstack().to_frame().T,age_stat.set_index("Activite").unstack().to_frame().T
    
    global_stat.loc[1]=list(age_stat.loc[0])
    
    global_stat.index=[equip[0]+" Potentiel global",equip[0]+" cible prioritaire "+ equip[1]]
    
    liste_df.append(global_stat)

In [358]:
## triatement des données pour la sortie du tableau consolidé

df= liste_df[0]

level_0=list(map(lambda x: x.split("_",maxsplit=1)[1],df.columns.levels[0]))

df.columns.set_levels(level_0,level=0,inplace=True)   ## Changer le label du niveau 0

stat_final=[df]

for stat_data in liste_df[1:]:
    
    stat_data.columns=df.columns
    
    stat_final.append(stat_data)

df_final=pd.concat(stat_final,ignore_index=False)

df_final=df_final.astype(int)

df_final

Cour                           \
Activite                                     Actif Autres Retraité    Total   
PIM Potentiel global                        572213  87418   671434  1331065   
PIM cible prioritaire [18 ans,36 ans]       106293  53491       26   159810   
RETRAITE Potentiel global                   530463  80944   612502  1223909   
RETRAITE cible prioritaire [18 ans,74 ans]  528917  76295   281743   886955   
ASSUR_VIE Potentiel global                  572176  81827   671029  1325032   
ASSUR_VIE cible prioritaire []              572176  81827   671029  1325032   
DECES Potentiel global                      550584  77175   671106  1298865   
DECES cible prioritaire [25 ans, 55ans]     388711  46908     2618   438237   
OBS Potentiel global                        568085  81290   633002  1282377   
OBS cible prioritaire [55 ans, 84 ans[      171278  12927   512942   697147   
CAM Potentiel global                        570393  81866   646613  1298872   
CAM cible prioritaire [55 ans, 74 ans[      171462  10652   267226   449340   
TOUT_HABITAT Potentiel global               467953  80926   650963  1199842   
TOUT_HABITAT cible prioritaire [0,70 ans[   464209  74749   151139   690097   
SANTE_ANIM Potentiel global                 574684  82148   671275  1328107   
SANTE_ANIM cible prioritaire []             574684  82148   671275  1328107   
PROTECT_JURIQ Potentiel global              574538  82136   671084  1327758   
PROTECT_JURIQ cible prioritaire []          574538  82136   671084  1327758   

                                              MAIL                           \
Activite                                     Actif Autres Retraité    Total   
PIM Potentiel global                         12967   3763     2820    19550   
PIM cible prioritaire [18 ans,36 ans]         6690   2766        3     9459   
RETRAITE Potentiel global                   495082  76584   430939  1002605   
RETRAITE cible prioritaire [18 ans,74 ans]  494373  73886   230001   798260   
ASSUR_VIE Potentiel global                  532952  77324   478615  1088891   
ASSUR_VIE cible prioritaire []              532952  77324   478615  1088891   
DECES Potentiel global                      510565  72477   478687  1061729   
DECES cible prioritaire [25 ans, 55ans]     367671  46168     2262   416101   
OBS Potentiel global                        529570  76950   448476  1054996   
OBS cible prioritaire [55 ans, 84 ans[      150361  10924   385955   547240   
CAM Potentiel global                        531643  77406   458339  1067388   
CAM cible prioritaire [55 ans, 74 ans[      151115   9525   221110   381750   
TOUT_HABITAT Potentiel global               436733  76468   460537   973738   
TOUT_HABITAT cible prioritaire [0,70 ans[   434614  72636   129383   636633   
SANTE_ANIM Potentiel global                 535510  77661   478840  1092011   
SANTE_ANIM cible prioritaire []             535510  77661   478840  1092011   
PROTECT_JURIQ Potentiel global              535372  77651   478662  1091685   
PROTECT_JURIQ cible prioritaire []          535372  77651   478662  1091685   

                                               SMS                          \
Activite                                     Actif Autres Retraité   Total   
PIM Potentiel global                          8772   3048     1515   13335   
PIM cible prioritaire [18 ans,36 ans]         4892   2294        2    7188   
RETRAITE Potentiel global                   468027  76166   371384  915577   
RETRAITE cible prioritaire [18 ans,74 ans]  467325  73578   204421  745324   
ASSUR_VIE Potentiel global                  501989  76845   410626  989460   
ASSUR_VIE cible prioritaire []              501989  76845   410626  989460   
DECES Potentiel global                      480064  72086   410703  962853   
DECES cible prioritaire [25 ans, 55ans]     349542  45966     2198  397706   
OBS Potentiel global                        498570  76464   382746  957780   
OBS cible prioritaire [55 

In [359]:
df_final.to_excel("Stat.xlsx",sheet_name="Potentiel OA par Canal")